# CS-GY 9223-E: Deep Learning Homework 1
Due on Sunday, 11th February 2018, 11:55 PM

This homework can be done in pairs. Everyone must submit on NYU Classes individually.

Write down the UNIs (NetIDs) of your group (if applicable)

Member 1: Name, NetID

Member 2: Name, NetID

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
# you shouldn't need to make any more imports

In [22]:
class NeuralNetwork(object):
    """
    Abstraction of neural network.
    Stores parameters, activations, cached values. 
    Provides necessary functions for training and prediction. 
    """
    def __init__(self, layer_dimensions, drop_prob=0.0, reg_lambda=0.0):
        """
        Initializes the weights and biases for each layer
        :param layer_dimensions: (list) number of nodes in each layer
        :param drop_prob: drop probability for dropout layers. Only required in part 2 of the assignment
        :param reg_lambda: regularization parameter. Only required in part 2 of the assignment
        """
        np.random.seed(1)
        
        self.parameters = {}
        self.num_layers = len(layer_dimensions)
        self.drop_prob = drop_prob
        self.reg_lambda = reg_lambda
        
        # init parameters
               
        for i in range(1, self.num_layers):            
            self.parameters["layer_" + str(i)] = np.random.randn(layer_dimensions[i], layer_dimensions[i-1] + 1) \
            * (2.0 / (layer_dimensions[i-1] * layer_dimensions[i])) ** 1/2
            print(self.parameters["layer_" + str(i)].shape)

        

    def affineForward(self, A, W, b):
        """
        Forward pass for the affine layer.
        :param A: input matrix, shape (L, S), where L is the number of hidden units in the previous layer and S is
        the number of samples
        :returns: the affine product WA + b, along with the cache required for the backward pass
        """
        print("****")
        print(A.shape)
        print(b.shape)
        print(W.shape)
#         A_b = np.hstack((b, A))
        A_b = np.vstack((b, A))
        affine_product = np.dot(W, A_b)
        
        return affine_product
        

    def activationForward(self, A, activation="relu"):
        """
        Common interface to access all activation functions.
        :param A: input to the activation function
        :param prob: activation funciton to apply to A. Just "relu" for this assignment.
        :returns: activation(A)
        """
        
        return self.relu(A)


    def relu(self, X):
        
        A = np.maximum(0, X)
        return A
    
    
    def softmax(self, Z):
        
        return np.exp(Z) / np.sum(np.exp(Z), axis=0)
    
            
    def dropout(self, A, prob):
        """
        :param A: Activation
        :param prob: drop prob
        :returns: tuple (A, M) 
            WHERE
            A is matrix after applying dropout
            M is dropout mask, used in the backward pass
        """
        
        M = np.random.rand(A.shape[0], A.shape[1])
        M = (M > prob) * 1.0
        scale_up_prob = 1 / (1 - prob)
        M *= scale_up_prob
        A *= M
        
        return A, M

    def forwardPropagation(self, X):
        """
        Runs an input X through the neural network to compute activations
        for all layers. Returns the output computed at the last layer along
        with the cache required for backpropagation.
        :returns: (tuple) AL, cache
            WHERE 
            AL is activation of last layer
            cache is cached values for each layer that
                     are needed in further steps
        """
        cache = []
        AL = []
        
#         n, m = X.shape 
        
#         X = np.hstack((np.ones((n,1)), X))
        
        X_forward = X
        for layer in self.parameters:
#             bias = np.ones((X_forward.shape[0],1))
            bias = np.ones((1,X_forward.shape[1]))
            print(layer)
            AL = self.activationForward(self.affineForward(X_forward, self.parameters[layer], bias))
            
            cache.append(AL)
            X_forward = AL
        
        print(AL.shape)
        print(len(cache))
        return AL, cache
    
    def costFunction(self, AL, y):
        """
        :param AL: Activation of last layer, shape (num_classes, S)
        :param y: labels, shape (S)
        :param alpha: regularization parameter
        :returns cost, dAL: A scalar denoting cost and the gradient of cost
        """
        # compute loss
        m = y.shape[0]
        AL_softmax = self.softmax(AL)
        correct_label_prob = AL_softmax[y, range(m)]
        cost = - np.sum(np.log(correct_label_prob)) / m
        
#         if self.reg_lambda > 0:
            # add regularization
       
        
        # gradient of cost
        dAL = 0
        return cost, dAL

#     def affineBackward(self, dA_prev, cache):
#         """
#         Backward pass for the affine layer.
#         :param dA_prev: gradient from the next layer.
#         :param cache: cache returned in affineForward
#         :returns dA: gradient on the input to this layer
#                  dW: gradient on the weights
#                  db: gradient on the bias
#         """

#         return dA, dW, db

#     def activationBackward(self, dA, cache, activation="relu"):
#         """
#         Interface to call backward on activation functions.
#         In this case, it's just relu. 
#         """

        
#     def relu_derivative(self, dx, cached_x):

#         return dx

#     def dropout_backward(self, dA, cache):

#         return dA

#     def backPropagation(self, dAL, Y, cache):
#         """
#         Run backpropagation to compute gradients on all paramters in the model
#         :param dAL: gradient on the last layer of the network. Returned by the cost function.
#         :param Y: labels
#         :param cache: cached values during forwardprop
#         :returns gradients: dW and db for each weight/bias
#         """
#         gradients = {}
        
#         for ...:
            
            
#             if self.drop_prob > 0:
#                 #call dropout_backward
           
            
#         if self.reg_lambda > 0:
#             # add gradients from L2 regularization to each dW
        
#         return gradients


#     def updateParameters(self, gradients, alpha):
#         """
#         :param gradients: gradients for each weight/bias
#         :param alpha: step size for gradient descent 
#         """

#     def train(self, X, y, iters=1000, alpha=0.0001, batch_size=100, print_every=100):
#         """
#         :param X: input samples, each column is a sample
#         :param y: labels for input samples, y.shape[0] must equal X.shape[1]
#         :param iters: number of training iterations
#         :param alpha: step size for gradient descent
#         :param batch_size: number of samples in a minibatch
#         :param print_every: no. of iterations to print debug info after
#         """
        
#         for i in range(0, iters):
#             # get minibatch
            
#             # forward prop

#             # compute loss

#             # compute gradients

#             # update weights and biases based on gradient

#             if i % print_every == 0:
#                 # print cost, train and validation set accuracies
                
    def predict(self, X):
        """
        Make predictions for each sample
        """
        AL, cache = NN.forwardPropagation(X)
        
#         y_pred = np.exp(AL) / np.sum(np.exp(AL), axis=0)
        
        return AL

#     def get_batch(self, X, y, batch_size):
#         """
#         Return minibatch of samples and labels
        
#         :param X, y: samples and corresponding labels
#         :parma batch_size: minibatch size
#         :returns: (tuple) X_batch, y_batch
#         """

#         return X_batch, y_batch

In [17]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [18]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    One-hot encoding converts categorical labels to binary values
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [19]:
# Load the data
data_root_path = '../HW1/cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  import sys


Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


## Part 1

#### Simple fully-connected deep neural network

In [20]:
# Parameters

learning_rate = 0.01
iterations = 10
batch_size = 128
print_every_set = 2

In [23]:
layer_dimensions = [X_train.shape[0], 20, 30, 40, 10]  # including the input and output layers
NN = NeuralNetwork(layer_dimensions)

NN.forwardPropagation(X_train)

(20, 3073)
(30, 21)
(40, 31)
(10, 41)
layer_1
****
(3072, 50000)
(1, 50000)
(20, 3073)
layer_2
****
(20, 50000)
(1, 50000)
(30, 21)
layer_3
****
(30, 50000)
(1, 50000)
(40, 31)
layer_4
****
(40, 50000)
(1, 50000)
(10, 41)
(10, 50000)
4


(array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.00057709,  0.00057709,  0.00057709, ...,  0.00057709,
          0.00057709,  0.00057709],
        ..., 
        [ 0.00023783,  0.00023783,  0.00023783, ...,  0.00023783,
          0.00023783,  0.00023783],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]),
 [array([[  5.15036295e-04,   6.35085593e-04,   2.93564863e-04, ...,
            3.75875534e-04,   3.95915507e-04,   4.27723809e-04],
         [  0.00000000e+00,   0.00000000e+00,   9.16322190e-05, ...,
            0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
         [  4.31909658e-04,   6.70819252e-05,   5.06697856e-04, ...,
            1.69778034e-04,   2.914

In [ ]:
NN.train(X_train, 
         y_train, 
         iters=iterations, 
         alpha=learning_rate, 
         batch_size=batch_size, 
         print_every=print_every_set)

In [24]:
y_predicted = NN.predict(X_test)
save_predictions('ans1-uni', y_predicted)

print(y_predicted)

layer_1
****
(3072, 10000)
(1, 10000)
(20, 3073)
layer_2
****
(20, 10000)
(1, 10000)
(30, 21)
layer_3
****
(30, 10000)
(1, 10000)
(40, 31)
layer_4
****
(40, 10000)
(1, 10000)
(10, 41)
(10, 10000)
4
[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.00057709  0.00057709  0.00057709 ...,  0.00057709  0.00057709
   0.00057709]
 ..., 
 [ 0.00023783  0.00023783  0.00023783 ...,  0.00023783  0.00023783
   0.00023783]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]


In [ ]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-uni.npy')
print(loaded_y.shape)
loaded_y[:10]

## Part 2: Regularizing the neural network
#### Add dropout and L2 regularization

In [ ]:
NN2 = NeuralNetwork(layer_dimensions, drop_prob=0, reg_lambda=0)
NN2.train(X_train, y_train, iters=1000, alpha=0.00001, batch_size=1000, print_every=10)

In [ ]:
y_predicted2 = NN2.predict(X)
save_predictions(y_predicted, 'ans2-uni')